In [ ]:
# fill namespace with numpy and matplotlib + static notebook plots
%pylab inline
# fill namespace with numpy and matplotlib + and interactive plots
#%pylab notebook 
#%pylab widget
#%matplotlib qt
import sk_dsp_comm.sigsys as ss
import sk_dsp_comm.digitalcom as dc
import sk_dsp_comm.rtlsdr_helper as sdrh
import sk_dsp_comm.fir_design_helper as fir_d
import sk_dsp_comm.iir_design_helper as iir_d
import ipywidgets as widgets
import imp # for module reloading
import scipy.signal as signal
from IPython.display import Audio, display
from IPython.display import Image, SVG

In [ ]:
pylab.rcParams['savefig.dpi'] = 100 # default 72
#pylab.rcParams['figure.figsize'] = (6.0, 4.0) # default (6,4)
#%config InlineBackend.figure_formats=['png'] # default for inline viewing
%config InlineBackend.figure_formats=['svg'] # SVG inline viewing
#%config InlineBackend.figure_formats=['pdf'] # render pdf figs for LaTeX
#Image('fname.png',width='80%')

# RTLSDR Stream Class

Now included in the `rtlsdr_helper` is the `RTLSDR_stream` class. This class contains async methods for reading IQ data from the rtl_sdr, decimating, and playing back audio.

In [ ]:
sdrh.pah.available_devices()

In [ ]:
sdr_stream = sdrh.RTLSDR_stream(0,rtl_buffer_size=2**15,audio_out=3)

# Mono FM Receiver Example

You can use pyaudio_helper to find available audio devices

In [ ]:
sdrh.pah.available_devices()

In [ ]:
sdr_stream = sdrh.RTLSDR_stream(0,rtl_buffer_size=2**15,audio_out=3)

In [ ]:
sdr_stream.interactive_FM_Rx(88.7e6,40,3,2048,48000)

In [ ]:
sdr_stream.show_logs()

In [ ]:
sdr_stream.clear_logs()

# User-Defined Callbacks

This callback will be called after the initial decimation 

In [ ]:
def callback(samples,fs,user_var):
    # Discriminator
    x = samples
    X=np.real(x)                # X is the real part of the received signal
    Y=np.imag(x)                # Y is the imaginary part of the received signal
    b=np.array([1, -1])         # filter coefficients for discrete derivative
    a=np.array([1, 0])          # filter coefficients for discrete derivative
    derY=signal.lfilter(b,a,Y)  # derivative of Y, 
    derX=signal.lfilter(b,a,X)  #    "          X,
    z_bb=(X*derY-Y*derX)/(X**2+Y**2)
    return z_bb,user_var

In [ ]:
sdr_stream.run_user_stream(callback,10,5)

In [ ]:
sdr_stream.set_audio_gain_db(0)

In [ ]:
sdr_stream.set_fc(103.9e6)

In [ ]:
sdr_stream.stop()

## Custom Filters

We can set the filter taps and initial conditions in the `run_user_stream()` method.

In [ ]:
M1 = 10
b = signal.firwin(32,2*200e3/float(2.4e6))
stage1_ic = signal.lfilter_zi(b,1)
M2 = 5
bb = signal.firwin(32,2*12e3/float(2.4e6)*M1)
stage2_ic = signal.lfilter_zi(bb,1)

In [ ]:
sdr_stream.run_user_stream(callback,M1,M2,b=b,stage1_ic=stage1_ic,bb=bb,stage2_ic=stage2_ic)

In [ ]:
sdr_stream.set_fc(88.7e6)

In [ ]:
sdr_stream.stop()

## Using the FIR Design Helper

In [ ]:
import sk_dsp_comm.fir_design_helper as fir_d

In [ ]:
fc = 200e3
trans = 50000
b = fir_d.fir_remez_lpf(fc-trans/2,fc+trans/2,1,40,2.4e6)
M1 = 10
M2 = 5
fc2 = 15e3
trans = 5000
bb = fir_d.fir_remez_lpf(fc2-trans/2,fc2+trans/2,1,40,2.4e6/10)
stage1_ic = signal.lfilter_zi(b,1)
stage2_ic = signal.lfilter_zi(bb,1)

In [ ]:
fir_d.freqz_resp_list([b],[1],'dB',fs=2.4e6,Npts=1024)
grid();

In [ ]:
fir_d.freqz_resp_list([bb],[1],'dB',fs=2.4e6/M1,Npts=1024)
grid();

In [ ]:
sdr_stream.run_user_stream(callback,M1,M2,b=b,stage1_ic=stage1_ic,bb=bb,stage2_ic=stage2_ic)

In [ ]:
sdr_stream.set_fc(103.9e6)

In [ ]:
sdr_stream.stop()

In [ ]:
fc = 200e3
taps = 16
b = fir_d.firwin_lpf(taps,fc,2.4e6)
M1 = 10
M2 = 5
fc2 = 14e3
bb = fir_d.firwin_lpf(taps,fc2,2.4e6/M1)
stage1_ic = signal.lfilter_zi(b,1)
stage2_ic = signal.lfilter_zi(bb,1)

In [ ]:
fir_d.freqz_resp_list([b],[1],'dB',fs=2.4e6,Npts=1024)
grid();

In [ ]:
fir_d.freqz_resp_list([bb],[1],'dB',fs=2.4e6/M1,Npts=1024)
grid();

In [ ]:
sdr_stream.run_user_stream(callback,10,5,b=b,stage1_ic=stage1_ic,bb=bb,stage2_ic=stage2_ic)

In [ ]:
sdr_stream.stop()

## Using IIR Design Helper

In [ ]:
import sk_dsp_comm.iir_design_helper as iir_d

In [ ]:
fc = 200e3
trans = 50000
b,a,sos = iir_d.IIR_lpf(fc,fc+trans,1,10,2.4e6,'butter')
M1 = 10
M2 = 5
fc2 = 15e3
trans = 5000
bb,aa,sos2 = iir_d.IIR_lpf(fc2,fc2+trans,1,10,2.4e6/M1,'butter')
stage1_ic = signal.lfilter_zi(b,a)
stage2_ic = signal.lfilter_zi(bb,aa)

In [ ]:
iir_d.freqz_resp_list([b],[a],fs=2.4e6,Npts=1024)
ylim(-80,10)
grid()

In [ ]:
iir_d.freqz_resp_list([bb],[aa],fs=2.4e6/M1,Npts=1024)
ylim(-80,10)
grid()

In [ ]:
sdr_stream.run_user_stream(callback,10,5,b,stage1_ic,a,bb,stage2_ic,aa)

In [ ]:
sdr_stream.set_fc(99.9e6)

In [ ]:
sdr_stream.stop()

In [ ]:
fc = 200e3
trans = 5000
b,a,sos = iir_d.IIR_lpf(fc,fc+trans,1,10,2.4e6,'cheby1')
M1 = 10
M2 = 5
fc2 = 17e3
trans = 500
bb,aa,sos2 = iir_d.IIR_lpf(fc2,fc2+trans,1,10,2.4e6/M1,'cheby1')
stage1_ic = signal.lfilter_zi(b,a)
stage2_ic = signal.lfilter_zi(bb,aa)

In [ ]:
iir_d.freqz_resp_list([b],[a],fs=2.4e6,Npts=1024)
ylim(-80,10)
grid()

In [ ]:
iir_d.freqz_resp_list([bb],[aa],fs=2.4e6/M1,Npts=1024)
ylim(-80,10)
grid()

In [ ]:
sdr_stream.run_user_stream(callback,10,5,b,stage1_ic,a,bb,stage2_ic,aa)

In [ ]:
sdr_stream.stop()

# Plotting

In [ ]:
sdr_stream.run_user_stream(callback,10,5)

In [ ]:
sdr_stream.set_fc(103.9e6)

In [ ]:
await sdr_stream.plot_rf(256)

In [ ]:
await sdr_stream.plot_stage1()

In [ ]:
await sdr_stream.plot_processed_stage1()

In [ ]:
await sdr_stream.plot_stage2()

In [ ]:
sdr_stream.stop()

## Pulling frames from stream

In [ ]:
sdr_stream.run_user_stream(callback,10,5)

In [ ]:
rf_frame = await sdr_stream.get_rf_frame()

In [ ]:
Nfft = 2**10
print(len(rf_frame))
Px,f = psd(rf_frame,Nfft,sdr_stream.get_sample_rate()/1e6,sdr_stream.get_center_freq()/1e6);

In [ ]:
stage1_frame = await sdr_stream.get_stage1_frame()

In [ ]:
print(len(stage1_frame))
psd(stage1_frame,Nfft,2.4e6,99.9e6/10);

In [ ]:
processed_stage1_frame = await sdr_stream.get_processed_stage1_frame()

In [ ]:
Nfft = 2**10
print(len(processed_stage1_frame))
psd(processed_stage1_frame,Nfft,2.4e6/10,0);

In [ ]:
stage2_frame = await sdr_stream.get_stage2_frame()

In [ ]:
Nfft = 2**10
print(len(stage2_frame))
psd(stage2_frame,Nfft,2.4e6/50,0);

In [ ]:
sdr_stream.stop()

# Spectrum Analyzer

In [ ]:
%pylab widget

In [ ]:
sdr_stream.set_audio_out(3)

In [ ]:
sdr_stream.run_user_stream(callback,10,5)

In [ ]:
sdr_stream.run_plot_rf_stream(1024,2,w=15,h=6)

In [ ]:
sdr_stream.set_fc(99.9e6)

In [ ]:
sdr_stream.set_NFFT(2048)

In [ ]:
sdr_stream.set_refresh_rate(4)

In [ ]:
sdr_stream.toggle_invert()

In [ ]:
sdr_stream.stop_rf_plot()

In [ ]:
sdr_stream.run_plot_stage1_stream(1024,1,w=15,h=6)

In [ ]:
sdr_stream.stop_stage1_plot()

In [ ]:
sdr_stream.run_plot_processed_stage1_stream(1024,1,w=15,h=6)

In [ ]:
sdr_stream.stop_processed_stage1_plot()

In [ ]:
sdr_stream.run_plot_stage2_stream(1024,1,w=15,h=6)

In [ ]:
sdr_stream.set_fc(99.9e6)

In [ ]:
sdr_stream.set_refresh_rate(4)

In [ ]:
sdr_stream.stop_stage2_plot()

In [ ]:
sdr_stream.stop_all()

# Using Widgets

In [ ]:
import ipywidgets as widgets

## Frequency Slider

In [ ]:
freq_slider = widgets.FloatSlider(
    value=103.9,
    min=87.5,
    max=108,
    step=0.2,
    description=r'$f_c\;\mathrm{(MHz)}$',
    continuous_update=True,
    orientation='horizontal',
    readout_format='0.1f',
    layout=widgets.Layout(
        width='90%',
    ) 
)
freq_slider.style.handle_color = 'lightblue'



In [ ]:
def set_freq_MHz(fc):
    sdr_stream.set_fc(fc*1e6)

In [ ]:
sdr_stream.run_user_stream(callback,10,5)

In [ ]:
center_freq_widget = widgets.interactive(set_freq_MHz, fc=freq_slider)
display(center_freq_widget)

In [ ]:
sdr_stream.run_plot_rf_stream(w=15,h=6)

In [ ]:
sdr_stream.stop_all()

## Audio Gain Slider

In [ ]:
audio_gain_slider = widgets.FloatSlider(
    value=-3,
    min=-60,
    max=6,
    step=0.1,
    description=r'Gain (dB)',
    continuous_update=True,
    orientation='horizontal',
    readout_format='0.1f',
    layout=widgets.Layout(
        width='90%',
    ) 
)
audio_gain_slider.style.handle_color = 'lightgreen'


In [ ]:
def set_audio_gain_db(db_gain):
    gain = 10**(db_gain/20)
    sdr_stream.set_audio_gain(gain)

In [ ]:
audio_gain_widget = widgets.interactive(set_audio_gain_db, db_gain=audio_gain_slider)

In [ ]:
display(audio_gain_widget)
display(center_freq_widget)

In [ ]:
sdr_stream.run_user_stream(callback,10,5)

In [ ]:
sdr_stream.stop()

## On/Off Toggle Buttons

In [ ]:
def radio_on_off(selection):
    if(selection == 'On'):
        def my_callback(samples,fs,user_val):
            # Discriminator
            x = samples
            X=np.real(x)                # X is the real part of the received signal
            Y=np.imag(x)                # Y is the imaginary part of the received signal
            b=np.array([1, -1])         # filter coefficients for discrete derivative
            a=np.array([1, 0])          # filter coefficients for discrete derivative
            derY=signal.lfilter(b,a,Y)  # derivative of Y, 
            derX=signal.lfilter(b,a,X)  #    "          X,
            z_bb=(X*derY-Y*derX)/(X**2+Y**2)
            return z_bb,user_val
        if(not sdr_stream.keep_streaming):
            sdr_stream.run_user_stream(my_callback,10,5)
    else:
        sdr_stream.stop()

In [ ]:
on_off = widgets.ToggleButtons(
    options=['On', 'Off'],
    description = ' ',
    value = 'Off'
)
on_off.style.button_width = "400px"
on_off.style.description_width = "1px"

In [ ]:
on_off_widget = widgets.interactive(radio_on_off,selection=on_off)

In [ ]:
display(on_off_widget)
display(audio_gain_widget)
display(center_freq_widget)

## Adjustable Stage 1 Filter

In [ ]:
stage1_fc = widgets.FloatSlider(
    value=100,
    min=2,
    max=200,
    step=0.1,
    description=r'$f_{cS1} \;\mathrm{(KHz)}$',
    continuous_update=False,
    orientation='horizontal',
    readout_format='0.1f',
    layout=widgets.Layout(
        width='90%',
    ) 
)
stage1_fc.style.handle_color = 'orange'

In [ ]:
def stage1_fc_change(fc):
    b = fir_d.firwin_lpf(64,2*fc*1e3,2.4e6)
    stage1_ic = signal.lfilter_zi(b,1)
    sdr_stream.set_stage1_coeffs(b,zi=stage1_ic)

In [ ]:
stage1_fc_widget = widgets.interactive(stage1_fc_change,fc=stage1_fc)

In [ ]:
display(on_off_widget)
display(audio_gain_widget)
display(stage1_fc_widget)
display(center_freq_widget)

In [ ]:
sdr_stream.run_plot_stage1_stream(1024,2,w=15,h=6)

In [ ]:
sdr_stream.stop_stage1_plot()

## Adjustable Stage 2 Filter

In [ ]:
stage2_fc = widgets.FloatSlider(
    value=12,
    min=2,
    max=20,
    step=0.1,
    description=r'$f_{cS2} \;\mathrm{(KHz)}$',
    continuous_update=False,
    orientation='horizontal',
    readout_format='0.1f',
    layout=widgets.Layout(
        width='90%',
    ) 
)
stage2_fc.style.handle_color = 'purple'

In [ ]:
def stage2_fc_change(fc):
    bb = fir_d.firwin_lpf(64,fc*1e3,2.4e6/10)
    stage2_ic = signal.lfilter_zi(bb,1)
    sdr_stream.set_stage2_coeffs(bb,zi=stage2_ic)

In [ ]:
stage2_fc_widget = widgets.interactive(stage2_fc_change,fc=stage2_fc)

In [ ]:
display(on_off_widget)
display(audio_gain_widget)
display(stage1_fc_widget)
display(stage2_fc_widget)
display(center_freq_widget)

In [ ]:
sdr_stream.run_plot_stage2_stream(1024,2,w=15,h=6)

In [ ]:
sdr_stream.stop_stage2_plot()

# Bypassing Audio

In [ ]:
import asyncio

In [ ]:
sdr_stream.set_rtl_buffer_size(16)

In [ ]:
def no_audio_callback(samples,fs,user_var):
    frame_count = user_var
    user_var = user_var+1
    return array([frame_count]),user_var

In [ ]:
global keep_collecting

In [ ]:
async def handle_data_out():
    global keep_collecting
    keep_collecting = True
    while keep_collecting:
        data_out = await sdr_stream.get_data_out_async()
        print(data_out)
    sdr_stream.reset_data_out_queue()
    print('Done')

In [ ]:
sdr_stream.run_user_stream(no_audio_callback,1,1,audio_sink=False,user_var=1)
task = asyncio.create_task(handle_data_out())

In [ ]:
keep_collecting = False

In [ ]:
sdr_stream.stop()